In [9]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [10]:
pandf = fdr.DataReader("KS11","2009.01.01","2019-01-01")
pandf

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2009-01-02,1157.40,1132.87,1157.40,1118.84,340780000.0,0.0293
2009-01-05,1173.57,1180.98,1185.22,1166.66,414760000.0,0.0140
2009-01-06,1194.28,1183.52,1203.59,1179.94,444200000.0,0.0176
2009-01-07,1228.17,1198.85,1228.56,1195.20,568560000.0,0.0284
2009-01-08,1205.70,1212.70,1218.43,1195.15,472340000.0,-0.0183
...,...,...,...,...,...,...
2018-12-21,2061.49,2052.70,2061.51,2049.76,311390000.0,0.0007
2018-12-24,2055.01,2050.38,2059.94,2046.18,285280000.0,-0.0031
2018-12-26,2028.01,2028.81,2037.83,2014.28,321500000.0,-0.0131


In [11]:
# convert nparray
nparr = pandf.values
nparr.astype('float32')
print(nparr)

[[ 1.15740e+03  1.13287e+03  1.15740e+03  1.11884e+03  3.40780e+08
   2.93000e-02]
 [ 1.17357e+03  1.18098e+03  1.18522e+03  1.16666e+03  4.14760e+08
   1.40000e-02]
 [ 1.19428e+03  1.18352e+03  1.20359e+03  1.17994e+03  4.44200e+08
   1.76000e-02]
 ...
 [ 2.02801e+03  2.02881e+03  2.03783e+03  2.01428e+03  3.21500e+08
  -1.31000e-02]
 [ 2.02844e+03  2.03209e+03  2.03557e+03  2.02139e+03  3.98020e+08
   2.00000e-04]
 [ 2.04104e+03  2.03670e+03  2.04697e+03  2.03541e+03  3.52680e+08
   6.20000e-03]]


In [12]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
nptf

array([[0.08774962, 0.08707471, 0.08125887, 0.07909785, 0.2815341 ,
        0.75432811],
       [0.09798782, 0.11720825, 0.09888968, 0.10908167, 0.3426833 ,
        0.62819456],
       [0.11110056, 0.11879917, 0.11053159, 0.11740842, 0.36701734,
        0.65787304],
       ...,
       [0.63898492, 0.64824372, 0.63922759, 0.64055152, 0.26559795,
        0.40478153],
       [0.63925718, 0.65029814, 0.63779533, 0.64500959, 0.32884662,
        0.51442704],
       [0.64723499, 0.6531856 , 0.64502003, 0.65380033, 0.29137021,
        0.56389118]])

In [13]:
print("d")

d


In [14]:
# split train, test

y=np.where(pandf["Change"].shift(-1).dropna()>0,1,0)
train_size = int(len(nptf) * 0.7)
y_train=y[:train_size]
y_test=y[train_size:]
X_train = nptf[:train_size]
X_test = nptf[train_size:]

In [15]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [16]:
# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(1, 6)))
for i in range(2):
    model.add(Dense(36,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.3)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1211 samples, validate on 520 samples
Epoch 1/100
1211/1211 [==============================] - 1s 455us/step - loss: 0.6936 - acc: 0.5029 - val_loss: 0.6948 - val_acc: 0.5019
Epoch 2/100
1211/1211 [==============================] - 0s 128us/step - loss: 0.6923 - acc: 0.5029 - val_loss: 0.6939 - val_acc: 0.5019
Epoch 3/100
1211/1211 [==============================] - 0s 107us/step - loss: 0.6920 - acc: 0.5227 - val_loss: 0.6934 - val_acc: 0.5019
Epoch 4/100
1211/1211 [==============================] - 0s 122us/step - loss: 0.6918 - acc: 0.5310 - val_loss: 0.6943 - val_acc: 0.4981
Epoch 5/100
1211/1211 [==============================] - 0s 118us/step - loss: 0.6906 - acc: 0.5227 - val_loss: 0.6920 - val_acc: 0.5231
Epoch 6/100
1211/1211 [==============================] - 0s 92us/step - loss: 0.6914 - acc: 0.5318 - val_loss: 0.6920 - val_acc: 0.5269
Epoch 7/100
1211/1211 [==============

1211/1211 [==============================] - 0s 130us/step - loss: 0.6893 - acc: 0.5367 - val_loss: 0.6966 - val_acc: 0.4981
Epoch 57/100
1211/1211 [==============================] - 0s 117us/step - loss: 0.6899 - acc: 0.5277 - val_loss: 0.6931 - val_acc: 0.5154
Epoch 58/100
1211/1211 [==============================] - 0s 135us/step - loss: 0.6895 - acc: 0.5417 - val_loss: 0.6921 - val_acc: 0.5269
Epoch 59/100
1211/1211 [==============================] - 0s 142us/step - loss: 0.6893 - acc: 0.5400 - val_loss: 0.6921 - val_acc: 0.5269
Epoch 60/100
1211/1211 [==============================] - 0s 117us/step - loss: 0.6893 - acc: 0.5516 - val_loss: 0.6922 - val_acc: 0.5212
Epoch 61/100
1211/1211 [==============================] - 0s 133us/step - loss: 0.6890 - acc: 0.5334 - val_loss: 0.6922 - val_acc: 0.5154
Epoch 62/100
1211/1211 [==============================] - 0s 120us/step - loss: 0.6899 - acc: 0.5425 - val_loss: 0.6921 - val_acc: 0.5250
Epoch 63/100
1211/1211 [=======================

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,np.where(model.predict(X_train)>0.5,1,0))

0.534950895436164

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        13872     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 48)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 48)        0         
______________________________________________

NameError: name 'train_len' is not defined

In [3]:
train_step(train_gen, test_gen, model)

NameError: name 'train_step' is not defined